In [1]:
class field:
    def __init__(self, width, height):
        self.surface = pg.surface.Surface(((width - width//3)*2, height*2))
        self.surface.fill((255,255,255))
        self.pool_objects = [list(), list()] # 0 - вентили, 1 - провода
        self.rect = self.surface.get_rect(center = (width//3, height//2))
        self.time_name = [0, -1] # хранит время последнего клика правой кнопки мыши и имя объекта
        self.moveobj = None
        self.old_coord = [0,0]
#        self.scale_val = 1
        self.coord_center = [width//3, height//2] # центр левой области
#        self.scale_speed = 0.1

        
    def redraw(self):
        self.surface.fill((255,255,255))
        for x in range(2):
            for element in self.pool_objects[x]:
                self.surface.blit(element.image, element.rect)
    
#    не лезь в скейл! оно тебя сожрёт
    def scale(self, k):# k - 1 - увеличение приближение, -1 - уменьшение
        new_scale = self.scale_val + self.scale_speed*k
        if 0.5 <= new_scale <= 1.5 and self.moveobj == None:
            for element in self.pool_objects[0]:
                element.rect.move_ip(int(round((new_scale/self.scale_val - 1)*(element.rect.center[0] - self.coord_center[0] + self.rect.left))),
                                    int(round((new_scale/self.scale_val - 1)*(element.rect.center[1] - self.coord_center[1] + self.rect.top))))
                dw_n = k*int(round(element.size[0]//2//int(round(0.5/self.scale_speed))))
                element.rect.inflate_ip(dw_n, int(round(dw_n*element.size[1]/element.size[0])))
                element.image = pg.transform.smoothscale(element.true_image, (element.rect.right - element.rect.left,
                                                                              element.rect.bottom - element.rect.top))

            self.old_coord = [self.coord_center[0] - self.rect.left, self.coord_center[1] - self.rect.top]
            ratio_nw2ow = 1/(self.rect.right - self.rect.left)
            self.rect.inflate_ip(k*self.coord_center[0]*2//int(round(0.5/self.scale_speed)),
                                    k*self.coord_center[1]*2//int(round(0.5/self.scale_speed)))
            if self.rect.right - self.rect.left < self.coord_center[0]*2 or self.rect.bottom - self.rect.top < self.coord_center[1]*2:
                self.rect.inflate_ip(self.coord_center[0]*2 - self.rect.right + self.rect.left,
                                    self.coord_center[1]*2 - self.rect.bottom + self.rect.top)
            self.surface = pg.transform.scale(self.surface, (self.rect.right - self.rect.left,
                                                            self.rect.bottom - self.rect.top))
            ratio_nw2ow *= self.rect.right - self.rect.left
            self.moveobj = self.rect
            self.move_obj((int(round(self.old_coord[0]*ratio_nw2ow)),
                           int(round(self.old_coord[1]*ratio_nw2ow))))
            self.scale_val = new_scale
            for element in self.pool_objects[0]:
                self.moveobj = element
                self.moveobj.update_offsets((0,0))
                self.move_obj((int(round(self.moveobj.rect.center[0]*ratio_nw2ow)),
                           int(round(self.moveobj.rect.center[1]*ratio_nw2ow))))
            self.moveobj = None
            self.redraw()
    
    def move_obj(self, coord):
        if self.moveobj == self.rect:
            self.rect.move_ip(coord[0] - self.old_coord[0], 
                              coord[1] - self.old_coord[1])
            if self.rect.left > 0:
                #self.rect.move_ip(-(coord[0] - self.old_coord[0]), 0)
                self.rect.move_ip(-self.rect.left, 0)
            elif self.rect.right < 2*self.coord_center[0]:
                #self.rect.move_ip(-(coord[0] - self.old_coord[0]), 0)
                self.rect.move_ip(2*self.coord_center[0] - self.rect.right, 0)
            if self.rect.bottom < 2*self.coord_center[1]:
                #self.rect.move_ip(0, -(coord[1] - self.old_coord[1]))
                self.rect.move_ip(0, 2*self.coord_center[1] - self.rect.bottom)
            elif self.rect.top > 0:
                #self.rect.move_ip(0, -(coord[1] - self.old_coord[1]))
                self.rect.move_ip(0, -self.rect.top)
            self.old_coord = coord
        else:    
            self.moveobj.move((coord[0] - self.rect.left,
                               coord[1] - self.rect.top)) 
        self.redraw()
    
    
    def click_lmouse(self, coord):
        for element in self.pool_objects[0][::-1]:
            if element.rect.collidepoint((coord[0] - self.rect.left, coord[1] - self.rect.top)):
                self.moveobj = element
                self.moveobj.update_offsets((coord[0] - self.rect.left, coord[1] - self.rect.top))
                self.pool_objects[0].remove(element)
                self.pool_objects[0].append(element)
                self.redraw()
                break
        if self.moveobj == None:
            self.moveobj = self.rect
            self.old_coord = [coord[0], coord[1]]
        
    def click_rmouse(self, coord):
        #self.del_object(pool_objects[1], coord)
        self.del_object(self.pool_objects[0], coord)
    
    def create_object(self, coord):
        self.pool_objects[0].append(Biba((coord[0] - self.rect.left, coord[1] - self.rect.top),
                                         path_rect, f"gate{len(self.pool_objects[0])}"))
#         dw_n = int(round((self.scale_val - 1)/self.scale_speed))*int(round(self.pool_objects[0][-1].size[0]//2//int(round(0.5/self.scale_speed))))
#         self.pool_objects[0][-1].rect.inflate_ip(dw_n, int(round(dw_n*self.pool_objects[0][-1].size[1]/self.pool_objects[0][-1].size[0])))
#         self.pool_objects[0][-1].image = pg.transform.smoothscale(self.pool_objects[0][-1].true_image, (self.pool_objects[0][-1].rect.right - self.pool_objects[0][-1].rect.left,
#                                                                               self.pool_objects[0][-1].rect.bottom - self.pool_objects[0][-1].rect.top))
        #self.surface.blit(self.pool_objects[0][-1].image, self.pool_objects[0][-1].rect)
        self.surface.blit(self.pool_objects[0][-1].image, self.pool_objects[0][-1].rect)
    def del_object(self, objs, coord):
        for element in objs[::-1]:
            if element.rect.collidepoint((coord[0] - self.rect.left, coord[1] - self.rect.top)):
                if abs(self.time_name[0] - time.time()) < 0.5 and self.time_name[1] == element.name:
                    objs.remove(element)
                    del element
                    self.time_name[0] = 0
                    self.redraw()
                    break
                else:
                    self.time_name[0] = time.time()
                    self.time_name[1] = element.name
                    break